In [1]:
import os
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
import os
import cv2
import random
import numpy as np
import torch
import argparse
from shutil import copyfile
from src.config import Config
from src.edge_connect import EdgeConnect

In [3]:
from src.models import EdgeModel, InpaintingModel
from load_config import load_config

In [4]:
input_folder = "Images/Inputs/"
output_folder = "Images/Outputs/"

In [5]:
config = load_config()
config.DEVICE = torch.device("cpu")

------------ Options -------------
INPUT_SIZE: 256
TEST_FLIST: ./Images/Inputs
RESULTS: ./Images/Outputs
MODE: 2
MODEL: 3
OBJECTS: [3, 15]
SEG_DEVICE: cpu
INPUT_SIZE: 256
-------------- End ----------------


In [6]:
# model = EdgeConnect(config)
# model.load()
edge_model = EdgeModel(config).to(config.DEVICE)
inpaint_model = InpaintingModel(config).to(config.DEVICE)

In [7]:
edge_model.load()
inpaint_model.load()

Loading EdgeModel generator...
Loading InpaintingModel generator...


In [8]:
edge_model.eval()

EdgeModel(
  (generator): EdgeGenerator(
    (encoder): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (9): ReLU(inplace=True)
    )
    (middle): Sequential(
      (0): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((2, 2, 2, 2))
          (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=(2, 2), bias=False)
          (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, aff

In [9]:
inpaint_model.eval()

InpaintingModel(
  (generator): InpaintGenerator(
    (encoder): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (9): ReLU(inplace=True)
    )
    (middle): Sequential(
      (0): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((2, 2, 2, 2))
          (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=(2, 2))
          (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine

# Load Data

In [10]:
def cuda(*args):
    return (item.to(config.DEVICE) for item in args)

In [11]:
def postprocess(img):
    # [0, 1] => [0, 255]
    img = img * 255.0
    img = img.permute(0, 2, 3, 1)
    return img.int()

In [12]:
from torch.utils.data import DataLoader
from src.dataset import Dataset

In [13]:
test_dataset = Dataset(config, config.TEST_FLIST, config.TEST_EDGE_FLIST, augment=False, training=False)

In [14]:
test_loader = DataLoader(
            dataset=test_dataset,
            batch_size=1,
        )

In [ ]:
index = 0
for items in test_loader:
    name = test_dataset.load_name(index)
        
    images, images_gray, edges, masks = cuda(*items)
    index += 1

D:\Python and Conda\envs\Object_Removal\lib\site-packages\torchvision\transforms\functional.py:132: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))


loading error: ./Images/Inputs\4.jpg


In [16]:
print(images.shape)
print(images_gray.shape)
print(edges.shape)
print(masks.shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])


In [17]:
edges = edge_model(images_gray, edges, masks)
outputs = inpaint_model(images, edges, masks)
outputs_merged = (outputs * masks) + (images * (1 - masks))

In [18]:
edges_result = postprocess(1 - edges)[0]
masked_result = postprocess(images * (1 - masks) + masks)[0]
output = postprocess(outputs_merged)[0]
print(edges_result.shape)

torch.Size([256, 256, 1])


In [ ]:
plt.subplot(1,3,1)
plt.imshow(edges_result)
?
plt.subplot(1,3,3)
plt.imshow(output)
plt.show()